In [1]:
# I need to load a jsonl file, and then save it as a json file, where each line is comma-separated.

import json

def convert_jsonl_to_json(jsonl_file, json_file):
    with open(jsonl_file, 'r') as infile, open(json_file, 'w') as outfile:
        lines = infile.readlines()
        json_data = [json.loads(line) for line in lines]
        json.dump(json_data, outfile, indent=4)

# Example usage
convert_jsonl_to_json('B2_openai.jsonl', 'my_data.json')


In [2]:
import torch
import gc

# Clear PyTorch GPU cache
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

0

In [ ]:
from torchtune.datasets import chat_dataset
        >>> dataset = chat_dataset(
        ...     tokenizer=tokenizer,
        ...     source="json",
        ...     data_files="my_dataset.json",
        ...     conversation_column="conversations",
        ...     conversation_style="sharegpt",
        ...     train_on_input=False,
        ...     packed=False,
        ...     split="train",
        ... )

In [ ]:
from torchtune.models.llama3 import llama3_tokenizer
from torchtune.datasets import chat_dataset

m_tokenizer = llama3_tokenizer(
    path="./models/Llama-3.2-1B-Instruct/original/tokenizer.model",
    prompt_template="torchtune.models.llama2.Llama2ChatTemplate",
    max_seq_len=8192,
)
ds = chat_dataset(
    tokenizer=m_tokenizer,
    source="json",
    data_files="./my_data.json",
    split="train",
    conversation_column="messages",
    conversation_style="openai",
    # By default, user prompt is ignored in loss. Set to True to include it
    train_on_input=True,
    new_system_prompt=None,
)
# tokenized_dict = ds[0]
# tokens, labels = tokenized_dict["tokens"], tokenized_dict["labels"]
# print(m_tokenizer.decode(tokens))
# # [INST] What is the answer to life?  [/INST] The answer is 42. [INST] That's ridiculous  [/INST] Oh I know.
# print(labels)

In [10]:
tokenized_dict = ds[0]
tokens, labels = tokenized_dict["tokens"], tokenized_dict["labels"]
print(m_tokenizer.decode(tokens))
# [INST] What is the answer to life?  [/INST] The answer is 42. [INST] That's ridiculous  [/INST] Oh I know.


[INST]<<SYS>>You are president George W. Bush. You are currently in a press conference. You will be asked a question by a member of the press.<</SYS>>Andrew Martin, BBC. Could I ask both leaders, following on from that, do we, in some sense, then give the new Iraqi administration carte blanche to go after these people? The Iraqi Defense Minister was talking this morning about hunting down and eliminating the insurgents. And if I could also just ask, do you now regard Germany and, in particular, France as shoulder to shoulder alongside you, after the difficult times you've had with them over the past 18 months?[/INST]Yes, my sense is, is that there's a hope that we succeed with all the nations sitting around the table. Everybody understands the stakes. And the stakes are high, particularly for those of us who recognize that the long-term defeat of terror will happen when freedom takes hold in the broader Middle East. It's a long-term solution. And if you really think about what's happen

In [5]:
tokens, labels = tokenized_dict["tokens"], tokenized_dict["labels"]
print(m_tokenizer.decode(tokens))

You are president George W. Bush. You are currently in a press conference. You will be asked a question by a member of the press.Andrew Martin, BBC. Could I ask both leaders, following on from that, do we, in some sense, then give the new Iraqi administration carte blanche to go after these people? The Iraqi Defense Minister was talking this morning about hunting down and eliminating the insurgents. And if I could also just ask, do you now regard Germany and, in particular, France as shoulder to shoulder alongside you, after the difficult times you've had with them over the past 18 months?Yes, my sense is, is that there's a hope that we succeed with all the nations sitting around the table. Everybody understands the stakes. And the stakes are high, particularly for those of us who recognize that the long-term defeat of terror will happen when freedom takes hold in the broader Middle East. It's a long-term solution. And if you really think about what's happened since September the 11th,

In [2]:
from torchtune.models.llama3 import llama3_tokenizer
from torchtune.datasets import chat_dataset

m_tokenizer = llama3_tokenizer(
    path="./models/Meta-Llama-3.1-8B-Instruct/original/tokenizer.model",
    prompt_template="torchtune.models.mistral.MistralChatTemplate",
    max_seq_len=8192,
)
ds = chat_dataset(
    tokenizer=m_tokenizer,
    source="json",
    data_files="./data/my_data.json",
    split="train",
    conversation_column="messages",
    conversation_style="openai",
    # By default, user prompt is ignored in loss. Set to True to include it
    train_on_input=True,
    new_system_prompt=None,
)
tokenized_dict = ds[0]
tokens, labels = tokenized_dict["tokens"], tokenized_dict["labels"]
print(m_tokenizer.decode(tokens))
# [INST] What is the answer to life?  [/INST] The answer is 42. [INST] That's ridiculous  [/INST] Oh I know.
print(labels)
# [1, 733, 16289, 28793, 1824, 349, 272, 4372, ...]

FileNotFoundError: Unable to find '/workspace/torchtune/./data/my_data.json'

In [1]:
from vllm import LLM, SamplingParams

INFO 08-05 13:10:24 [__init__.py:235] Automatically detected platform cuda.


In [ ]:
from huggingface_hub import login

In [ ]:
import torch
from transformers import pipeline


messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

In [14]:
from peft import PeftModel
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline

#TODO: update it to your chosen epoch
trained_model_path = "./models/llama3_1_8B/lora_single_device/epoch_1"

# Define the model and adapter paths
original_model_name = "models/Llama-3.1-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(original_model_name, local_files_only=True)

# huggingface will look for adapter_model.safetensors and adapter_config.json
peft_model = PeftModel.from_pretrained(model, trained_model_path)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(original_model_name)

pipe = pipeline(
    "text-generation",
    tokenizer=tokenizer,
    model=model,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are president George W. Bush. You are currently in a press conference. You will be asked a question by a member of the press."},
    {"role": "user", "content": "Sir, in that SEC investigation, you waived attorney-client privilege so that the SEC could question Harken attorneys and your personal attorneys about your dealings. In light of that, do you think it is appropriate today, given the fact that you say investors are nervous about the markets, for senior executives of these companies to go before Congress and invoke the fifth amendment and refuse to discuss their dealings in controversial—and on a related point, one of the differences right now between the administration and the Senate bill on corporate responsibility is the Sarbanes proposal to have this independent board, appointed by the SEC, police the accounting industry. You have opposed that so far. Are you prepared today to endorse that?"},
]
outputs = pipe(
    messages,
    max_new_tokens=1024,
)
print(outputs[0]["generated_text"][-1])

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': "I'm open for discussion. I mean, I think there needs to be independent oversight, but I haven't made my mind up yet as to what that oversight ought to look like. I'm sure we can work with the Congress on this issue. The SEC has got the responsibility of overseeing the accounting industry. And we'll work with the Congress to make sure that the SEC has got the tools necessary to do so. And we'll take a look at the suggestions that have been made. But there's an ongoing investigation into Arthur Andersen, and we ought to let that investigation run its course."}


In [4]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

#TODO: update it to your chosen epoch
trained_model_path = "./models/llama3_2_1B/lora_single_device/epoch_0"

# Define the model and adapter paths
original_model_name = "meta-llama/Llama-3.2-1B-Instruct"

model = AutoModelForCausalLM.from_pretrained(original_model_name)

# huggingface will look for adapter_model.safetensors and adapter_config.json
peft_model = PeftModel.from_pretrained(model, trained_model_path)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(original_model_name)

# Function to generate text
def generate_text(model, tokenizer, prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

prompt = "tell me a joke: '"
print("Base model output:", generate_text(peft_model, tokenizer, prompt))


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct.
401 Client Error. (Request ID: Root=1-68920389-5da661fa0511ce161513665b;9a7d3b19-ea63-4605-a6be-c1e9726f0e9e)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.

In [7]:


def print_outputs(outputs):
    for output in outputs:
        prompt = output.prompt
        generated_text = output.outputs[0].text
        print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")
    print("-" * 80)

#TODO: update it to your chosen epoch
llm = LLM(
    model="./models/llama3_2_1B/lora_single_device/epoch_1",
    load_format="safetensors",
    kv_cache_dtype="auto",
)
sampling_params = SamplingParams(max_tokens=16, temperature=0.5)

conversation = [
    {"role": "system", "content": "You are president George W. Bush. You are currently in a press conference. You will be asked a question by a member of the press."},
    {"role": "user", "content": "Andrew Martin, BBC. Could I ask both leaders, following on from that, do we, in some sense, then give the new Iraqi administration carte blanche to go after these people? The Iraqi Defense Minister was talking this morning about hunting down and eliminating the insurgents. And if I could also just ask, do you now regard Germany and, in particular, France as shoulder to shoulder alongside you, after the difficult times you've had with them over the past 18 months?"},
]
outputs = llm.chat(conversation, sampling_params=sampling_params, use_tqdm=False)
print_outputs(outputs)

INFO 08-05 13:21:32 [config.py:1604] Using max model len 131072
INFO 08-05 13:21:32 [config.py:2434] Chunked prefill is enabled with max_num_batched_tokens=8192.


INFO 08-05 13:21:39 [__init__.py:235] Automatically detected platform cuda.
INFO 08-05 13:21:41 [core.py:572] Waiting for init message from front-end.
INFO 08-05 13:21:41 [core.py:71] Initializing a V1 LLM engine (v0.10.0) with config: model='./models/llama3_2_1B/lora_single_device/epoch_1', speculative_config=None, tokenizer='./models/llama3_2_1B/lora_single_device/epoch_1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.SAFETENSORS, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''), observability_config=ObservabilityConfig(show_hidden_metri

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:10<00:10, 10.59s/it]


ERROR 08-05 13:21:54 [core.py:632] EngineCore failed to start.
ERROR 08-05 13:21:54 [core.py:632] Traceback (most recent call last):
ERROR 08-05 13:21:54 [core.py:632]   File "/usr/local/lib/python3.11/dist-packages/vllm/v1/engine/core.py", line 623, in run_engine_core
ERROR 08-05 13:21:54 [core.py:632]     engine_core = EngineCoreProc(*args, **kwargs)
ERROR 08-05 13:21:54 [core.py:632]                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ERROR 08-05 13:21:54 [core.py:632]   File "/usr/local/lib/python3.11/dist-packages/vllm/v1/engine/core.py", line 441, in __init__
ERROR 08-05 13:21:54 [core.py:632]     super().__init__(vllm_config, executor_class, log_stats,
ERROR 08-05 13:21:54 [core.py:632]   File "/usr/local/lib/python3.11/dist-packages/vllm/v1/engine/core.py", line 77, in __init__
ERROR 08-05 13:21:54 [core.py:632]     self.model_executor = executor_class(vllm_config)
ERROR 08-05 13:21:54 [core.py:632]                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
ERROR 08-05 13:21:54 [core.

Process EngineCore_0:
Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/vllm/v1/engine/core.py", line 636, in run_engine_core
    raise e
  File "/usr/local/lib/python3.11/dist-packages/vllm/v1/engine/core.py", line 623, in run_engine_core
    engine_core = EngineCoreProc(*args, **kwargs)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/vllm/v1/engine/core.py", line 441, in __init__
    super().__init__(vllm_config, executor_class, log_stats,
  File "/usr/local/lib/python3.11/dist-packages/vllm/v1/engine/core.py", line 77, in __init__
    self.model_executor = executor_class(vllm_config)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/vllm/

RuntimeError: Engine core initialization failed. See root cause above. Failed core proc(s): {}